In [7]:
import torch
torch.__version__

'1.13.0.dev20220721+cu113'

FSDP currently does not support layer level fine tuning.  Thus options are whole model fine tuning, 
or for large language models, Child Tuning. 

Child Tuning was developed in the paper: Raise a Child in Large Language Model: Towards Effective and Generalizable Fine-tuning


In [8]:
from torch.distributions import Bernoulli

In [43]:
torch.manual_seed(2022)
torch.cuda.manual_seed(2022)

In [53]:
grad = torch.randn(3,3)
reserve_p = .30

In [54]:
grad

tensor([[ 2.0203,  0.1361, -0.9314],
        [ 1.3920,  0.7097, -2.1463],
        [ 0.9796,  0.2208, -0.3193]])

In [46]:
r = grad.new_full(size=grad.size(), fill_value=reserve_p)
r


tensor([[0.3000, 0.3000, 0.3000],
        [0.3000, 0.3000, 0.3000],
        [0.3000, 0.3000, 0.3000]])

In [47]:
rdist = Bernoulli(r)
rdist

Bernoulli(probs: torch.Size([3, 3]))

In [48]:
rp = rdist.sample() 
rp

tensor([[0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 1.]])

In [49]:
amplifier = rp/reserve_p
amplifier

tensor([[0.0000, 3.3333, 0.0000],
        [0.0000, 0.0000, 3.3333],
        [3.3333, 0.0000, 3.3333]])

In [50]:
newgrad = grad*amplifier
newgrad

tensor([[ 0.0000,  1.1021,  0.0000],
        [ 0.0000, -0.0000, -3.0270],
        [-7.7042,  0.0000,  6.8715]])

In [25]:
grad_mask = Bernoulli(grad.new_full(size=grad.size(), fill_value=reserve_p))
grad *= grad_mask.sample() / reserve_p

AttributeError: 'Bernoulli' object has no attribute 'data'

In [23]:
grad

tensor([[-0.8473, -0.0000,  0.0000],
        [-0.0000,  0.0000,  2.7251],
        [-0.0000, -0.5592,  0.0000]])